<!--
Ignore ShapelyDeprecationWarning warning in render

In [ ]:
import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning)

-->

# 11 Reprojecting

In this section we will learn how to join dataframes and will apply this to creating a [choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) with `geopandas`.

## About the data

The first dataset we will use is a [list of Arctic communities and their location](https://search.dataone.org/view/doi%3A10.18739%2FA28S4JQ80) @brook_approximate_2023 created by the [Alaska Native Tribal Health Consortium](https://www.anthc.org). This data comes in a GeoJSON file with the following attributes:

- **name**: name of Arctic community, 
- **population**: population of Arctic community, as of 2022
- **country**: country that the Arctic community falls within (see dataset metadata for the codes)
- **geoname-id**: numeric codes that uniquely identify all administrative/legal and statistical geographic areas for which the Census Bureau tabulates data

The second dataset is [Natural Earth's medium-scale  cultural boundaries data for countries (1:50m)](https://www.naturalearthdata.com/downloads/50m-cultural-vectors/). 
We can obtain this dataset by downloading the shapefile. 
[Natural Earth](https://www.naturalearthdata.com) is a public domain dataset with free, ready-to-use data for creating maps. 

The third dataset we will use is a CSV file with the country codes and names of the Arctic countries in the Arctic communities dataset. This dataset was created for educational purposes for this lesson based on the metadata of the Arctic communities dataset and the country names in Natural Earth's dataset. It can be accessed [here]().

## Import data

We will first import the countries shapefile and adapt it for wrangling purposes:

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

# Import countries polygons
fp = os.path.join('data', 'ne_50m_admin_0_countries', 'ne_50m_admin_0_countries.shp')
countries = gpd.read_file(fp)

# Simplify column names
countries.columns = countries.columns.str.lower()

# Select columns for analysis
countries = countries[['admin', 'type', 'geometry']]

countries.head()

In [ ]:
# Quick view
countries.plot()

Next, we import the Arctic communities data. Similar to how we previously used `pandas.read_csv()`, we can read in the Arctic communities GeoJSON data directly from the data repository using `geopandas.read_file()`:

In [ ]:
# Import Arctic communities data
URL = 'https://cn.dataone.org/cn/v2/resolve/urn%3Auuid%3Aed7718ae-fb0d-43dd-9270-fbfe80bfc7a4'
communities = gpd.read_file(URL)

communities.head()

The CRS of the `communities` is EPSG:4326. Remember all GeoJSON files are given in this CRS and all points are expressed in longitude and latitude units of decimal degrees. In this case, the `countries` and `communities` GeoDataFrames both have the same CRS:

In [ ]:
print(countries.crs)

Since the CRSs match, it is easy to take a quick look at our communities data by plotting it on top of the countries dataframe:

In [ ]:
# Verify CRSs match
assert countries.crs == communities.crs

fig, ax = plt.subplots()
countries.plot(ax=ax)
communities.plot(ax=ax, color='red')
plt.show()

Finally, we import the country names and codes CSV:

In [ ]:
country_names = pd.read_csv(os.path.join('data','country_names.csv'))
country_names

## Arctic communities by country

Next, we want to calculate the number of Arctic communities by country. 

In [ ]:
# Create data frame with number of communities per country
n_comms = (communities.groupby('country')
                      .size()
                      .reset_index(name='n_communities'))

Let's break this down:

- We start with our `communities` dataframe and use `groupby('country')` to group by country code.
- Then we use [`size()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.size.html) as an aggregator function to calculate the size of each group.
- The result of this operation is a `pandas.Series` indexed by the `country` values.
- By resetting the index we transform the `pandas.Series` into a `pandas.DataFrame`, the index is now a column named `country` and the values of the series are named `n_communities`.

In [ ]:
# Number of Arctic communities per country
n_comms

## `if`-`else` statements

Our goal is to merge the `n_comms` and the `countries` data frames. To merge two data frames they need to have at least one column in common. 
Currently our datasets do not have any columns in common:

In [ ]:
set(countries.columns).intersection(n_comms.columns)

The output `set()` represents the empty set. This might not be as informative, so let's write a different information statement:

In [ ]:
common_columns = set(countries.columns).intersection(n_comms.columns)

# Check if there are any common columns
if len(common_columns) != 0:
    print(f"Common columns: {common_columns}")
else:
    print("No common columns")

Remember that an `if-else` statement is a control structure that allows code to make decisions: it checks a condition, and if that condition is true, it executes one block of code (the `if` block); if the condition is false, it executes a different block (the `else` block). This enables programs to respond differently depending on specific criteria or inputs.

:::{.callout-tip}
## Check-in
Wrap up the previous code into a function called `check_common_columns` that prints a message depending of whether two data frames have common columns or not. Don't forget to include a docstring!
<!--

In [ ]:
def check_common_columns(df1, df2):
    """
    Prints a message depending whether df1 and df2 have columns in common

     Parameters:
    - df1: The first DataFrame.
    - df2: The second DataFrame.
    
    Returns:
    - A set of common column names. If no columns are common, returns an empty set.
    """

    common_columns = set(df1.columns).intersection(df2.columns)
    
    if common_columns:
        print(f"Common columns: {common_columns}")
    else:
        print("No common columns")
    
    return 

-->
:::

## Merging data frames

We can use the `country_names` data frame to add the names countries into the `n_comms` data which, in turn, will allow us to merge that data frame with the `country_names` data. To merge dataframes we can use the [`pandas.merge()` function](https://pandas.pydata.org/docs/reference/api/pandas.merge.html#pandas.merge). 
The basic syntax for it is:

```python
output_df = pd.merge(left_df,
                     right_df, 
                     how = type_of_join, 
                     on = column_to_join)
```
where

- `output_df` is the dataframe resulting from the merge,
- `left_df` is the dataframe we have "on the left side",
- `right_df` is the dataframe we have "on the right side",
- `how` specifies the type of join between the left and right dataframes, ([check the options here](https://pandas.pydata.org/docs/reference/api/pandas.merge.html#pandas.merge)), the default is to do an inner join,
- `on` specifies the column to join on, this column must be present in both our dataframes. 

So, we merge the `n_comms` and `country_names` data frames using a left join:

![Image source: Data Modeling Essentials, NCEAS Learning Hub @do-linh_open_2023](/book/images/merging_data/join-diagrams-left.png)

In [ ]:
n_comms = pd.merge(n_comms,
                   country_names,
                   how='left',
                   on='country')
n_comms

We can reuse our function to check that `n_comms` and `countries` now have a common column on which we can merge them:

In [ ]:
check_common_columns(n_comms, countries)

:::{.callout-tip}
## Check-in
Perform an inner join between our `countries` and `n_comms` dataframes. This will merge the subset of rows that have matches in both the left table and the right table.

![Image source: Data Modeling Essentials, NCEAS Learning Hub @do-linh_open_2023](/book/images/merging_data/join-diagrams-inner.png)
<!--

In [ ]:
# Merge data frames 
arctic_countries = pd.merge(countries,
                            n_comms,
                            how='inner',
                            on='admin')
# Update index
arctic_countries = arctic_countries.set_index('admin')

-->
:::

## Reviewing results

Notice that the row for Aland Islands is not present in the merged dataframe:

In [ ]:
arctic_countries

The `values` attribute of a data frame returns *all* the values in the data frame as an array. We can verify the value 'Aland Islands' was *nowhere* in our original countries dataframe like this:

In [ ]:
# Check Aland Islands is nowhere in data frame
'Aland Islands' not in countries.values

The Aland Islands is an autonomous region of Finland and there is one Arctic community registered in this region. 
We will directly add one to Finland to not lose this piece of data:

In [ ]:
arctic_countries.at['Finland', 'n_communities'] += 1

## Choropleth map

A [choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) is an efficient way to visualize aggregate data per region. 

Making a choropleth map from our polygons `GeoDataFrame` is easy; we just need to specify the `column` parameter in `plot()` and make it equal to the column with the values we want to plot in each country:

In [ ]:
arctic_countries.plot(column='n_communities',
                      legend=True)

## Reprojecting

Remember that CRSs reflect cultural views and even biases. Any map projection involves choices about which areas to emphasize, minimize, or distort, and those choices can influence how viewers perceive different regions. In our map, using the EPSG:4326 CRS is, among other things, mapping the Arctic regions as far apart, while they are actually near each other.

**Reprojecting** means changing the coordinate reference system of your geospatial data. Do
In our case, we will reproject the Alaska geo-dataframe to the CRS EPSG:3413. 
This CRS is a *projected* CRS, better suited for working with data from the Arctic region:

![Source: [spatialreference.org](https://spatialreference.org/ref/epsg/3413/) ](/book/images/lesson-12/epsg_3413.png)

Changing CRSs in GeoPandas is very simple using the `to_crs()` method for `geopandas.GeoDataFrame`s. 
The general syntax is:
```python
updated_geodf = geodf.to_crs(new_crs)
```
where:

- `updated_geodf` is the output of the method, a new geodataframe (`to_crs()` does not work in place), 
- `geodf` is the `geopandas.GeoDataFrame` we want to transform, 
- `new_crs` the CRS we want to convert to, this is an object of type CRS or string representing the CRS (ex: `'epsg:3413'`). 

In our case:

In [ ]:
# Reproject to CRS optimized for Arctic region
arctic_countries = arctic_countries.to_crs('epsg:3413')

We can now use the reprojected data to update our map:

In [ ]:
#| code-fold: true
#| 
fig, ax = plt.subplots(figsize=(8, 6))

# Remove the axis for a cleaner map
ax.axis('off')

# Create choropleth map of communities
# Plot with refined color and edge style
arctic_countries.plot(
    ax=ax,
    column='n_communities',
    cmap='PuBuGn',
    legend=True,
    edgecolor="0.6",
    linewidth=0.5,
    legend_kwds={
        "shrink": 0.7,
        "label": "Number of Arctic Communities",
        "orientation": "horizontal",
        "pad": 0.05
    }
)

# Add title and subtitle for better context
ax.set_title('Distribution of Arctic Communities', fontsize=18, weight='bold', pad=15)

plt.show()

Although the new projection clearly improves the presentation of the data, there are still issues with this plot! Mainly, the entire United States territory is in it, when we should only have Alaska. **In our next lesson we will review startegies to clip and subset vector data and return to this plot in our discussion section.**